In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import os

# Set up the WebDriver
driver = webdriver.Chrome()

# Open the page
driver.get("https://fostac.fssai.gov.in/traineeFBOReg")
time.sleep(3)

# Select the state dropdown
State = driver.find_element(By.XPATH, """/html/body/div[2]/form/fieldset/div[6]/div[1]/div[2]/div/div/select""")
State.click()
time.sleep(2)

# Get all state options (for dynamic state selection)
state_options = State.find_elements(By.TAG_NAME, "option")

# Specify the CSV file name
csv_file = "districts_and_cities.csv"

# Check if the CSV file exists
file_exists = os.path.exists(csv_file)

# Loop through all the state options (excluding the first one which is "Select State")
for i, state_option in enumerate(state_options[1:], 1):
    state_name = state_option.text.strip()
    
    # Select the current state
    state_option.click()
    time.sleep(2)

    # Select the district dropdown
    District = driver.find_element(By.XPATH, """/html/body/div[2]/form/fieldset/div[6]/div[2]/div[1]/div/div/select""")
    District.click()
    time.sleep(2)

    # Get all the district options under the selected state
    select_tag = driver.find_element(By.XPATH, """/html/body/div[2]/form/fieldset/div[6]/div[2]/div[1]/div/div/select""")
    district_options = select_tag.find_elements(By.TAG_NAME, "option")

    # Initialize an empty list to store districts and cities
    districts_and_cities = []

    # Iterate over the options to extract district names
    for district_option in district_options:
        district_name = district_option.text.strip()
        
        # Skip the '--Not Selected--' option
        if district_name != '--Not Selected --':
            # Select the current district
            district_option.click()
            time.sleep(2)

            # Now select the City dropdown
            City = driver.find_element(By.XPATH, """/html/body/div[2]/form/fieldset/div[6]/div[2]/div[2]/div/div/select""")
            City.click()
            time.sleep(2)

            # Get all city options under the selected district
            city_options = City.find_elements(By.TAG_NAME, "option")

            # If no city is selected, we use "None" for the city
            if city_options:
                for city_option in city_options:
                    city_name = city_option.text.strip()
                    if city_name != '--Not Selected --':
                        districts_and_cities.append((district_name, state_name, city_name))
            else:
                # If no city is selected, set city to None
                districts_and_cities.append((district_name, state_name, "None"))

    # Parse the districts, cities, and state into a DataFrame
    df = pd.DataFrame(districts_and_cities, columns=["District", "State", "City"])

    # Append the DataFrame to the CSV file
    df.to_csv(csv_file, mode='a', header=not file_exists, index=False)

    # Set file_exists to True after first write, to avoid header rewrite on subsequent appends
    file_exists = True
